In [9]:
#read https://github.com/neilbalch/SimVascular-pythondemos/tree/master/Testing%20Framework
import os
import sys

SV_PATH       = '/home/marsdenlab/projects/SV/fork/SimVascular/Python/site-packages'
SV_BUILD_PATH = '/home/marsdenlab/projects/SV/fork/build/SimVascular-build/lib'

sys.path.append(SV_PATH)

#chdir needed to find sv shared object files (.so)
os.chdir(SV_BUILD_PATH)

import sv
print("\n sv")
print(dir(sv))

print("\n sv Geom")
print(dir(sv.Geom))

print("\n sv Solid")
print(dir(sv.Solid))

print("\n sv SolidPolyData")
print(dir(sv.Solid.SolidPolyData))

print("\n sv MeshTetgen")
print(dir(sv.MeshTetgen))

print("\n sv MeshUtil")
print(dir(sv.MeshUtil))


 sv
['CircleContour', 'Contour', 'Geom', 'Image', 'Itkls', 'LevelSetContour', 'Math', 'MeshAdapt', 'MeshObject', 'MeshTetgen', 'MeshUtil', 'Path', 'PolygonContour', 'Repository', 'Solid', 'SolidOCCT', 'SolidPolyData', 'SplinePolygonContour', 'TetGenAdapt', 'ThresholdContour', 'VMTKUtils', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'ctypes', 'ext', 'func', 'myDLL', 'myDll', 'sys']

 sv Geom
['2dWindingNum', 'AddPointData', 'AlignProfile', 'All_union', 'AvgPt', 'Bbox', 'Checksurface', 'Classify', 'Clean', 'Copy', 'DisorientProfile', 'DividePointData', 'FindDistance', 'GetClosedLineRegion', 'GetOrderedPts', 'GetPolyCentroid', 'IntegrateEnergy', 'IntegrateScalarSurf', 'IntegrateScalarThresh', 'IntegrateSurface2', 'IntegrateSurfaceFlux', 'InterpolateScalar', 'InterpolateVector', 'Intersect', 'IntersectWithLine', 'Local_blend', 'Local_butterfly_subdivision', 'Local_constrain_smooth', 'Local_decimation', 'Local_laplac

In [3]:
sys.platform

'linux'